## CS585: Natural Language Processing
### Hidden Markov Models
<br><br>
#### Illinois Institute of Technology  
#### Aron Culotta

<br><br><br><br><br>

## Last week
- Language models
- Naive Bayes classification

## This week

- Hidden Markov models
- Like a language model, but with labels attached to each token


<br><br><br>

#### Example applications of Hidden Markov models

- Part-of-speech tagging:

| Det | N   | V      | P    | D   | N    |
|-----|-----|--------|------|-----|------|
| The | cow | jumped | over | the | moon. |


- Named-entity recognition

| Person    | Person|  _     | _    | Location  | _    |
|-----------|-------|--------|------|-----------|------|
| President | Trump | flew   | to   | D.C.      | today. |

- Speech recognition

| "hu"      | "ow" |  "ah"  | "r"  | "ye"      | "ooh"    |
|-----------|------|--------|------|-----------|----------|
|-----------|------|--------|------|-----------|----------|

<img src='figs/audio.jpg' width=50%/>

All of the above examples have a sequence of input symbols (e.g., words) and a sequence of output symbols (e.g., parts-of-speech).

<br><br><br>

#### The main idea of Hidden Markov models

The output at time $i$ depends on the input at time $i$ and the output at previous times $i-1$, $i-2$, ...:

- **Language model**: $p(w_i|w_{i-2}, w_{i-1})$
- **Hidden Markov model**: $p(y_i|w_i, y_{i-1}, y_{i-2}, \ldots)$
  - e.g., $y_i$ is part-of-speech tag at time $i$

<br><br><br>

## Markov chain


To begin, let's consider an **observed** Markov chain.

The language models from last week is an example of this.  
E.g, bigram language model assumes next word is independent of all other words given the previous word:

$$p(w_i \mid w_{i-n} \ldots w_{i-1}) \triangleq p(w_i \mid w_{i-1})$$

This is a **first-order Markov assumption**. We can draw this model as a **weighted** finite state machine.

E.g., assume our language model only has three words  
{"snow", "white", "is"}:

![snow](figs/snow.png)

- $p($is $ \mid $ snow, white$)  \triangleq   p($ is $ \mid $ snow$) = a_{21}$
- Each edge is weighted by a conditional probability.





## Markov Chain

A Markov chain consists of the following:

- $Q=q_1 \ldots q_N$, a set of $N$ **states**
- $A = a_{01}a_{02} \ldots a_{n1} \ldots a_{nn}$, a **transition probability matrix**.
  - Each $a_{ij}=p(s_j | s_i)$ is the probability of the transition from state $i$ to state $j$
  - All transitions from a state must sum to 1: $\sum_{j=1}^n a_{ij} = 1 $ $\forall i$
- $q_0$: start state
- $q_F$: end (final) state


#### A Markov chain assigns a probability to a sequence of words
- Equal to the product of the probabilities for an accepting path
  - If multiple accepting paths, equal to the largest value for any path
  
  

![snow](figs/snow.png)

$p($"snow white is"$)$    
$= p($snow $\mid q_0) * p($ white $\mid $ snow$ ) * p($ is $\mid $ white$) * p(q_F \mid $ is $)$  
$= a_{02} * a_{23} * a_{31} * a_{14}$


#### "Unrolled" chain

![snow_unrolled](figs/snow_unrolled.png)





## Hidden Markov Model (HMM)

- A Markov chain over **unobserved** ("hidden") variables

![pos](figs/pos.png)

![pos_u](figs/pos_unrolled.png)


## Eisner's "weather example", adapted

- Future climatologist wants to know:
  - What was the weather like in Chicago in Summer 2016?
- We do not have records of temperature, but we luckily Mayor Rahm Emanuel kept records of the number of ice creams he ate each day.

**Problem:**

- Given a sequence of observations $O$
  - ints representing number of ice creams Rahm ate per day
- Predict the hidden sequence $Q$ of weather states
  - "H" for Hot or "C" for Cold
  
![icecream](figs/icecream.png)

## HMMs, formally

A Hidden Markov Model consists of the following:

- $Q=q_1 \ldots q_n$, a set of $n$ **states**
- $A = a_{01}a_{02} \ldots a_{n1} \ldots a_{nn}$, a **transition probability matrix**.
  - Each $a_{ij}=p(s_j | s_i)$ is the probability of the transition from state $i$ to state $j$
  - All transitions from a state must sum to 1: $\sum_{j=1}^n a_{ij} = 1 $ $\forall i$
- $O = o_1 \ldots o_T$, a sequence of $T$ **observations**
  - Each is drawn from vocabulary $V = v_1 \ldots v_V$
- $B = b_i(o_i)$, a sequence of **observation likelihoods**, aka **emission probabilities**
  - The probability of observation $o_i$ being generated by state $i$
  - $p(o_i|q_i)$
- $q_0$: start state and $q_F$: end (final) state
  - Neither is associated with observations
  - Each has transition probabilities for states:
    - $a_{01} \ldots a_{0n}$ for start transitions
    - $a_{1F} \ldots a_{nF}$ for end transitions

### HMM Assumptions

1. **Markov assumption**:
  - $p(q_i \mid q_1 \ldots q_{i-1}) \triangleq p(q_i \mid q_{i-1})$  
<br><br>
2. **Output independence**:
  - $p(o_i \mid q_1 \ldots q_i \ldots q_T, o_1 \ldots o_i \ldots o_T) \triangleq p(o_i \mid q_i)$
  
  
<br><br>
Why make these assumptions?


<br><br><br>

HMMs may be:
- **fully connected** ("ergodic"): every state is reachable from every other state
- or not: e.g., illegal to transition from "of" to present tense verb ("of jump")

## Three Fundamental Problems of HMMs

1. **Likelihood:**
  - Given an HMM $\lambda = (A,B)$ and an observation sequence $O$
  - Compute the likelihood $p(O \mid \lambda)$.
  - I.e., what is the probability of this observation sequence given this HMM?
<br><br><br>  
2. **Decoding:**
  - Given an observation sequence $O$ and an HMM $\lambda=(A,B)$
  - Find the most probable sequence $Q$
<br><br><br>
3. **Learning:**
  - Given an observation sequence $O$ and the set of states in the HMM
  - Learn the HMM parameters $A$ and $B$.
  
<br><br><br><br>

#### 1. Computing Likelihood with the Forward Algorithm

 Given an HMM $\lambda = (A,B)$ and an observation sequence $O$
  - Compute the likelihood $p(O \mid \lambda)$.
  - I.e., what is the probability of this observation sequence given this HMM?
  
<br><br>
Recall how we computed the likelihood of an *observed* sequence in a **Markov chain**
- Just multiply transition probabilities

$p($"snow white is"$)$    
$= p($snow $\mid q_0) * p($ white $\mid $ snow$ ) * p($ is $\mid $ white$) * p(q_F \mid $ is $)$  
$= a_{02} * a_{23} * a_{31} * a_{14}$

![snow_unrolled](figs/snow_unrolled.png)

<br><br>

##### Why doesn't this work for HMMs?

![pos_u](figs/pos_unrolled.png)

<br><br><br>
We don't observe the hidden states!

<br><br>
Let's start with a simpler problem:
- Assume we know the hidden states for this observation sequence
- E.g., in ice cream example
  - Rahm ate {3, 1, 3} ice creams
  - The weather was {hot, hot, cold}
  - $O=\{3,1,3\}$
  - $Q = \{H, H, C\}$
  
<br><br>
We can compute probability as:

$$
p(O|Q) = \prod_i^T p(o_i \mid q_i)
$$

$$p(\{3, 1, 3\} \mid \{H, H, C\}) = p(3 \mid H) * p(1 \mid H) * p(3 \mid C)$$  
$$= .4 * .2 * .1 $$

![l1](figs/likelihood1.png)


<br>

Since we don't know the true weather state sequence, we will sum over all possible state sequences, weighted by probability.

First, note that the joint probability $p(O,Q)$ just multiplies all the transitions $A$ and $B$:
$$
p(O, Q) = p(O \mid Q) \times p(Q) = \prod_i^T p(o_i \mid q_i) \times \prod_i^T p(q_i \mid q_{i-1})
$$

![icj](figs/icecream_joint.png)

To compute $p(O)$ from $p(O, Q)$, recall the notion of *marginalization* in probability:

$$p(x=x_j) = \sum_{y_i} p(x=x_j,y=y_i)$$  

E.g., if $x=1$ means a student passed test 1, and $y=1$ means a student passed test 2,  
then the probability that a student passes test 1 is:

$$
p(x=1) = p(x=1,y=0) + p(x=1,y=1)
$$

<br>

We will use marginalization to sum over possible state sequences, to compute the probability of the observation sequence.

$$
p(O) = \sum_Q p(O, Q) = \sum_Q p(O \mid Q) p(Q)
$$
- last step by chain rule: $p(X,Y) = p(X \mid Y) p(Y)$  


<br>
Back to the ice cream example:

$$ p(\{3, 1, 3\}) = p(\{3, 1, 3\}, \{C, C, C\}) + p(\{3, 1, 3\}, \{C, C, H\}) + \ldots $$  
$$ + p(\{3, 1, 3\}, \{H, H, C\}) + p(\{3, 1, 3\}, \{H, H, H\}) $$


**Problem with this approach**: If we have $N$ states and $T$ observations, how many possible hidden sequences must we sum over?

<br><br><br><br>
$$N^T$$

<br><br><br>
Is there a polynomial time algorithm?
<br><br>
Yes: Dynamic programming to the rescue again.
- There is a lot of duplicate work in the summation above.
- We will store intermediate values as we scan the input from left to right.
<br><br>


## Forward algorithm

##### Forward Trellis
Stores observation probabilities up to time step $t$

- $\alpha_t(j)=$ the probability of being in state $j$ after seeing the first $t$ observations  
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$=p(o_1\ldots o_t, q_t=j \mid \lambda) = \sum_{i=1}^N \alpha_{t-1}(i)a_{ij}b_j(o_t)$
  - $\alpha_{t-1}(i)$ the previous forward path probability
  - $a_{ij}$ the transition probability from previous state $i$ to current state $j$
  - $b_j(o_t)$ the likelihood of the observation at time $t$ $p(o_t \mid q_j$)
<br><br>
- E.g., $\alpha_2(2)$ computes forward probability of being in state 2 at time 2 having generated the partial observation $\{3, 1\}$
  - Generate two next steps:
    - $\alpha_1(1) \times p(H \mid H) \times p(1 \mid H)$ (if previous state was $H$)
    - $\alpha_1(2) \times p(H \mid C) \times p(1 \mid C)$ (if previous state was $C$)
    
![trellis](figs/trellis.png)


##### Recursive definition

1. Initialization:
  - $\alpha_1(j) = a_{0j}b_j(o_j) $ &nbsp; $1 \le j \le N$
  - probabilities of starting in state $j$ and emitting observation $o_1$
<br><br>
2. Recursion
  - $\alpha_t(j) = \sum_{i=1}^N \alpha_{t-1}(i)a_{ij}b_j(o_t) $ &nbsp;&nbsp;$ 1 \le j \le N, 1 < t < T$
  - the probability of being in state $j$ after seeing the first $t$ observations
  - sums over all possible states we could have been in at time step $t-1$
<br><br>
3. Termination
  - $ p(O \mid \lambda) = \alpha_T(q_F) = \sum_{i=1}^N \alpha_T(i)a_{iF}$ 
  
![trellis2](figs/trellis2.png)

#### Pseudocode

![code](figs/forward_code.png)

**Runtime?**
<br><br><br><br>
$$O(N^2T) << O(N^T)$$

<br><br><br><br>


## Decoding with the Viterbi algorithm

`2`. **Decoding:**
  - Given an observation sequence $O$ and an HMM $\lambda=(A,B)$
  - Find the most probable sequence $Q$
  - E.g., given the ice cream observations $\{3, 1, 3\}$, what is the most likely temperature sequence?
  
  
**Naive approach:**  
- Run the forward algorithm for each possible state sequence
- Return the state sequence that maximizes $p(O \mid \lambda)$
- Still exponential: $O(N^T)$

**Efficient approach:**

#### Viterbi algorithm

A dynamic program, similar to the forward algorithm, to find best path through the trellis.

![viterbi](figs/viterbi.png)

$$v_t(j) = \max_{q_0 \ldots q_{t-1}}p(q_0\ldots q_{t-1}, o_1 \ldots o_t, q_t = j \mid \lambda)$$  
$$ = \max_{i=1}^N v_{t-1}(i)a_{ij}b_j(o_t)$$
- $v_{t-1}(i)$: the previous Viterbi path probability from the prior time step
- $a_{ij}$: the transition probability from prior state $q_i$ to current state $q_j$
- $b_j(o_t)$: the probability of the observation at time $t$: $p(o_t \mid q_t)$

Contrast with the forward algorithm: $\alpha_t(j)=\sum_{i=1}^N \alpha_{t-1}a_{ij}b_j(o_t)$  
Two differences:
 - sum becomes $\max$
 - Also need additional bookkeeping to store the best path itself, rather than just the probability.






##### Pseudocode

![vit_code](figs/viterbi_code.png)

##### Recursive definition

1. Initialization:
  - $v_1(j) = a_{0j}b_j(o_j) $ &nbsp; $1 \le j \le N$
    - probabilities of starting in state $j$ and emitting observation $o_1$
  - $bt_1(j) = 0$ back trace to store best path
<br><br>
2. Recursion
  - $v_t(j) = \max_{i=1}^N v_{t-1}(i)a_{ij}b_j(o_t) $ &nbsp;&nbsp;$ 1 \le j \le N, 1 < t < T$
    - the probability of the best path continuing to state $j$ after seeing the first $t$ observations
  - $bt_t(j) = argmax_{i=1}^N v_{t-1}(i)a_{ij}b_j(o_t) $ &nbsp;&nbsp;$ 1 \le j \le N, 1 < t < T$
    - update the best path so far
<br><br>
3. Termination
  - $ P^* = v_T(q_F) = \max_{i=1}^N v_T(i)a_{iF}$ 
    - the best score
  - $q_T^* = bt_T(q_F) = argmax_{i=1}^N v_T(i) * a_{iF}$
    - the start of the backtrace (the best second to last state)
  

### Next time:

- Learning parameters $A$ and $B$ from data.

Also see assignment 2.

#### image sources
- https://www.cs.colorado.edu/~martin/SLP/


In [1]:
from IPython.core.display import HTML
HTML(open('../custom.css').read())